# Lesson 6: Expanding the SEC Knowledge Graph

### Environment variables
Sets environment variables. If asked, please replace the following [your-project-id] with your project ID and run it.

In [1]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]

ACCESS_TOKEN = !gcloud auth print-access-token
ACCESS_TOKEN = ACCESS_TOKEN[0]

EMBEDDING_MODEL = "textembedding-gecko@002"

LOCATION = "us-central1"

if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

In [2]:
PROJECT_ID

'gpeg-oe-platform'

In [3]:
ACCESS_TOKEN

'ya29.c.c0AY_VpZhzRWZ9a1GnxPxwH92r1ATRH3JuOuwX7AYjBwGX6x_jOfyHuJhccg2hTeOz0Gdb1I_jSRaDaYD48Tk281gs1c4G1_nLJWW3O3jzVCA1HHSnUy513fS60ai5vocJJFzF8wI-fGZOZuUA_lCPGB3ehn92tLaJ6beWJO95Py3BBSP01uxpBZgNSCbEirgxXlAjR9VT-HLIPILpIh5197PIv4MEh05M-fJC1XL_2EP0Yh1deG8pp1bUSyIfiqzYEtIJ4DmcaKAH0Om2KN0EReOikJGV946ACodjWJ2jU52FU_YsJVvrnq490fT-mqOm6zRVvP5fbg5cInJEcCN8K-CLbmvZ0zVaF-_BDc5aylbaaSL1CD51AiSpb1JZz0rVGfAT396Aw2xrUYYjXlY0Y9sgZ_Z00fnmfShIcsOk3-6zF2aJosF1zu3r9OJvFSqfIlBxk283eer6eswgSi4aaRWwry_vW4ziZ83i5frjIUa1uuJwvaetyMv0ipVXIk0B-bywRpRXw2_4i9qIhjYqx06wtui-8pJ36YclVdmJS54Fm5X2Uekret8o8g8h4Vmwz-RMxooOd7bBZyJx5ejdZYb2Zd5a9alum0vRw4co4whVRs0_SciRlM2iRsYOUfxezXBzlpqJrbuk-Xxt8--_vOh3ko5F1ohfi6ysx3vwzIkIj7OFFvt3FBMdx2pVxgb-I88o9Sd_MXJ4t4af3Bd75aFRxwBlajvlRlb3q1sjmX3lkk8S91b9zoawVesRq9VM7tsVYrRuflg373BZd-RmOw08rslmIWqde4wu_V8ouXgod9pcB5QyBfF0xwpRO9YvSdOa8Sxwk-hM0SqsOy5F-p1pYIIu7F1Vo1s6fSBzfso0RY84Yc3gh5uy5_uifByt4Fpvt4Q5z4ub5ysoht_VZmdYlpR4Qb1VOj-vj98BYygO6Sbq-WJdpkWvUqgrQnpsY2xJB9XjSe_nkB43UWmdbQf-0__Rr6

### Authentication (Colab only)
If you are running this notebook on Colab, you will need to run the following cell authentication. This step is not required if you are using Vertex AI Workbench as it is pre-authenticated.

In [4]:
import sys

# if it's Colab runtime, authenticate the user with Google Cloud
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

<p style="background-color:#fd4a6180; padding:15px; margin-left:20px"> ⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️<b>Note:</b> This notebook takes about 30 seconds to be ready to use. Please wait until the "Kernel starting, please wait..." </p>


### Import packages and set up Neo4j

In [5]:
# init the vertexai package
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [6]:
from dotenv import load_dotenv
import os
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain

from langchain_community.llms import VertexAI
from langchain_google_vertexai import VertexAIEmbeddings

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [7]:
# Load from environment
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'

# Global constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [8]:
kg = Neo4jGraph(
    url=NEO4J_URI, 
    username=NEO4J_USERNAME, 
    password=NEO4J_PASSWORD, 
    database=NEO4J_DATABASE
)

### Read the collection of Form 13s
- Investment management firms must report on their investments in companies to the SEC by filing a document called **Form 13**
- You'll load a collection of Form 13 for managers that have invested in NetApp
- You can check out the CSV file by navigating to the data directory using the File menu at the top of the notebook

In [9]:
import csv

all_form13s = []

with open('./data/form13.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader: # each row will be a dictionary
      all_form13s.append(row)

- Look at the contents of the first 5 Form 13s

In [10]:
all_form13s[0:5]

[{'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
  'managerCik': '1000275',
  'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
  'managerName': 'Royal Bank of Canada',
  'reportCalendarOrQuarter': '2023-06-30',
  'cusip6': '64110D',
  'cusip': '64110D104',
  'companyName': 'NETAPP INC',
  'value': '64395000000.0',
  'shares': '842850'},
 {'source': 'https://sec.gov/Archives/edgar/data/1002784/0001387131-23-009542.txt',
  'managerCik': '1002784',
  'managerAddress': '1875 Lawrence Street, Suite 300, Denver, CO, 80202-1805',
  'managerName': 'SHELTON CAPITAL MANAGEMENT',
  'reportCalendarOrQuarter': '2023-06-30',
  'cusip6': '64110D',
  'cusip': '64110D104',
  'companyName': 'NETAPP INC',
  'value': '2989085000.0',
  'shares': '39124'},
 {'source': 'https://sec.gov/Archives/edgar/data/1007280/0001007280-23-000008.txt',
  'managerCik': '1007280',
  'managerAddress': '277 E TOWN ST, COLUMBUS, OH, 43215',
  'managerName': 'PUBLIC 

In [12]:
len(all_form13s)

561

### Create company nodes in the graph
- Use the companies identified in the Form 13s to create `Company` nodes
- For now, there is only one company - NetApp

In [56]:
# work with just the first form fow now
first_form13 = all_form13s[0]

cypher = """
MERGE (com:Company {cusip6: $cusip6})
  ON CREATE
    SET com.companyName = $companyName,
        com.cusip = $cusip
"""

kg.query(cypher, params={
    'cusip6':first_form13['cusip6'], 
    'companyName':first_form13['companyName'], 
    'cusip':first_form13['cusip'] 
})

[]

In [57]:
cypher = """
MATCH (com:Company)
RETURN com LIMIT 1
"""

kg.query(cypher)

[{'com': {'cusip': '64110D104',
   'names': ['Netapp Inc', 'NETAPP INC'],
   'companyName': 'NETAPP INC',
   'cusip6': '64110D'}}]

- Update the company name to match Form 10-K

In [58]:
cypher = """
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  RETURN com.companyName, form.names
"""

kg.query(cypher)

[{'com.companyName': 'NETAPP INC', 'form.names': ['Netapp Inc', 'NETAPP INC']}]

In [59]:
cypher = """
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  SET com.names = form.names
"""

kg.query(cypher)

[]

- Create a `FILED` relationship between the company and the Form-10K node

In [60]:
kg.query("""
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  MERGE (com)-[:FILED]->(form)
""")


[]

### Create manager nodes
- Create a `manager` node for companies that have filed a Form 13 to report their investment in NetApp
- Start with the single manager who filed the first Form 13 in the list

In [61]:
cypher = """
  MERGE (mgr:Manager {managerCik: $managerParam.managerCik})
    ON CREATE
        SET mgr.managerName = $managerParam.managerName,
            mgr.managerAddress = $managerParam.managerAddress
"""

kg.query(cypher, params={'managerParam': first_form13})

[]

In [62]:
kg.query("""
  MATCH (mgr:Manager)
  RETURN mgr LIMIT 1
""")

[{'mgr': {'managerCik': '1423442',
   'managerAddress': '6 Suburban Avenue, Stamford, CT, 06901-2012',
   'managerName': "O'SHAUGHNESSY ASSET MANAGEMENT, LLC"}}]

- Create a uniquness constraint to avoid duplicate managers

In [63]:
kg.query("""
CREATE CONSTRAINT unique_manager 
  IF NOT EXISTS
  FOR (n:Manager) 
  REQUIRE n.managerCik IS UNIQUE
""")

[]

- Create a fulltext index of manager names to enable text search

In [64]:
kg.query("""
CREATE FULLTEXT INDEX fullTextManagerNames
  IF NOT EXISTS
  FOR (mgr:Manager) 
  ON EACH [mgr.managerName]
""")


[]

In [65]:
kg.query("""
  CALL db.index.fulltext.queryNodes("fullTextManagerNames", 
      "royal bank") YIELD node, score
  RETURN node.managerName, score
""")

[{'node.managerName': 'Royal Bank of Canada', 'score': 3.7019896507263184},
 {'node.managerName': 'Royal Fund Management, LLC',
  'score': 2.2780675888061523},
 {'node.managerName': 'ROYAL LONDON ASSET MANAGEMENT LTD',
  'score': 2.063053607940674},
 {'node.managerName': 'AMALGAMATED BANK', 'score': 1.798886775970459},
 {'node.managerName': 'COMMERCE BANK', 'score': 1.798886775970459},
 {'node.managerName': 'UMB Bank, n.a.', 'score': 1.5895916223526},
 {'node.managerName': 'Norinchukin Bank, The', 'score': 1.5895916223526},
 {'node.managerName': 'HUNTINGTON NATIONAL BANK', 'score': 1.5895916223526},
 {'node.managerName': 'Swiss National Bank', 'score': 1.5895916223526},
 {'node.managerName': 'AMERICAN NATIONAL BANK', 'score': 1.5895916223526},
 {'node.managerName': 'TORONTO DOMINION BANK', 'score': 1.5895916223526},
 {'node.managerName': 'Bangor Savings Bank', 'score': 1.5895916223526},
 {'node.managerName': 'Czech National Bank', 'score': 1.5895916223526},
 {'node.managerName': 'DEUTS

- Create nodes for all companies that filed a Form 13

In [66]:
cypher = """
  MERGE (mgr:Manager {managerCik: $managerParam.managerCik})
    ON CREATE
        SET mgr.managerName = $managerParam.managerName,
            mgr.managerAddress = $managerParam.managerAddress
"""
# loop through all Form 13s
for form13 in all_form13s:
  kg.query(cypher, params={'managerParam': form13 })

In [67]:
kg.query("""
    MATCH (mgr:Manager) 
    RETURN count(mgr)
""")

[{'count(mgr)': 561}]

### Create relationships between managers and companies
- Match companies with managers based on data in the Form 13
- Create an `OWNS_STOCK_IN` relationship between the manager and the company
- Start with the single manager who filed the first Form 13 in the list

In [68]:
cypher = """
  MATCH (mgr:Manager {managerCik: $investmentParam.managerCik}), 
        (com:Company {cusip6: $investmentParam.cusip6})
  RETURN mgr.managerName, com.companyName, $investmentParam as investment
"""

kg.query(cypher, params={ 
    'investmentParam': first_form13 
})

[{'mgr.managerName': 'Royal Bank of Canada',
  'com.companyName': 'NETAPP INC',
  'investment': {'shares': '842850',
   'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
   'managerName': 'Royal Bank of Canada',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'value': '64395000000.0',
   'cusip6': '64110D',
   'cusip': '64110D104',
   'reportCalendarOrQuarter': '2023-06-30',
   'companyName': 'NETAPP INC',
   'managerCik': '1000275'}}]

In [70]:
cypher = """
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik}), 
        (com:Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN { 
    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter
}]->(com)
ON CREATE
    SET owns.value  = toFloat($ownsParam.value), 
        owns.shares = toInteger($ownsParam.shares)
RETURN mgr.managerName, owns.reportCalendarOrQuarter, com.companyName
"""

kg.query(cypher, params={ 'ownsParam': first_form13 })

[{'mgr.managerName': 'Royal Bank of Canada',
  'owns.reportCalendarOrQuarter': '2023-06-30',
  'com.companyName': 'NETAPP INC'}]

In [71]:
kg.query("""
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik})
-[owns:OWNS_STOCK_IN]->
        (com:Company {cusip6: $ownsParam.cusip6})
RETURN owns { .shares, .value }
""", params={ 'ownsParam': first_form13 })

[{'owns': {'shares': 842850, 'value': 64395000000.0}}]

- Create relationships between all of the managers who filed Form 13s and the company

In [72]:
cypher = """
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik}), 
        (com:Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN { 
    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter 
    }]->(com)
  ON CREATE
    SET owns.value  = toFloat($ownsParam.value), 
        owns.shares = toInteger($ownsParam.shares)
"""

#loop through all Form 13s
for form13 in all_form13s:
  kg.query(cypher, params={'ownsParam': form13 })

In [73]:
cypher = """
  MATCH (:Manager)-[owns:OWNS_STOCK_IN]->(:Company)
  RETURN count(owns) as investments
"""

kg.query(cypher)

[{'investments': 561}]

In [74]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties: Chunk {chunkId: STRING, names: LIST,
formId: STRING, cik: STRING, cusip6: STRING, source: STRING,
f10kItem: STRING, chunkSeqId: INTEGER, text: STRING,
textEmbedding: LIST} Form {names: LIST, formId: STRING, cik:
STRING, cusip6: STRING, source: STRING} Company {names:
LIST, cusip6: STRING, companyName: STRING, cusip: STRING}
Manager {managerCik: STRING, managerName: STRING,
managerAddress: STRING} Relationship properties: SECTION
{f10kItem: STRING} OWNS_STOCK_IN {reportCalendarOrQuarter:
STRING, value: FLOAT, shares: INTEGER} The relationships:
(:Chunk)-[:NEXT]->(:Chunk) (:Chunk)-[:PART_OF]->(:Form)
(:Form)-[:SECTION]->(:Chunk) (:Company)-[:FILED]->(:Form)
(:Manager)-[:OWNS_STOCK_IN]->(:Company)


### Determine the number of investors
- Start by finding a form 10-K chunk, and save to use in subsequent queries

In [75]:
cypher = """
    MATCH (chunk:Chunk)
    RETURN chunk.chunkId as chunkId LIMIT 1
    """

chunk_rows = kg.query(cypher)
print(chunk_rows)

[{'chunkId': '0000950170-23-027948-item1-chunk0002'}]


In [76]:
chunk_first_row = chunk_rows[0]
print(chunk_first_row)

{'chunkId': '0000950170-23-027948-item1-chunk0002'}


In [77]:
ref_chunk_id = chunk_first_row['chunkId']
ref_chunk_id

'0000950170-23-027948-item1-chunk0002'

- Build up path from Form 10-K chunk to companies and managers

In [78]:
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form)
    RETURN f.source
    """

kg.query(cypher, params={'chunkIdParam': ref_chunk_id})

[{'f.source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}]

In [79]:
cypher = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
    (com:Company)-[:FILED]->(f)
RETURN com.companyName as name
"""

kg.query(cypher, params={'chunkIdParam': ref_chunk_id})

[{'name': 'NETAPP INC'}]

In [80]:
cypher = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
        (com:Company)-[:FILED]->(f),
        (mgr:Manager)-[:OWNS_STOCK_IN]->(com)
RETURN com.companyName, 
        count(mgr.managerName) as numberOfinvestors 
LIMIT 1
"""

kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})

[{'com.companyName': 'NETAPP INC', 'numberOfinvestors': 561}]

### Use queries to build additional context for LLM
- Create sentences that indicate how much stock a manager has invested in a company

In [81]:
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
        (com:Company)-[:FILED]->(f),
        (mgr:Manager)-[owns:OWNS_STOCK_IN]->(com)
    RETURN mgr.managerName + " owns " + owns.shares + 
        " shares of " + com.companyName + 
        " at a value of $" + 
        apoc.number.format(toInteger(owns.value)) AS text
    LIMIT 10
    """
kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})

[{'text': "O'SHAUGHNESSY ASSET MANAGEMENT, LLC owns 5656 shares of NETAPP INC at a value of $432,118,000"},
 {'text': 'CAPSTONE INVESTMENT ADVISORS, LLC owns 13800 shares of NETAPP INC at a value of $1,054,320,000'},
 {'text': 'Royal Bank of Canada owns 842850 shares of NETAPP INC at a value of $64,395,000,000'},
 {'text': 'SHELTON CAPITAL MANAGEMENT owns 39124 shares of NETAPP INC at a value of $2,989,085,000'},
 {'text': 'PUBLIC EMPLOYEES RETIREMENT SYSTEM OF OHIO owns 106941 shares of NETAPP INC at a value of $8,170,000'},
 {'text': 'WILBANKS SMITH & THOMAS ASSET MANAGEMENT LLC owns 6617 shares of NETAPP INC at a value of $505,539,000'},
 {'text': 'DEPRINCE RACE & ZOLLO INC owns 320581 shares of NETAPP INC at a value of $24,492,389,000'},
 {'text': 'COMMERCE BANK owns 101422 shares of NETAPP INC at a value of $7,748,640,000'},
 {'text': 'D. E. Shaw & Co., Inc. owns 323440 shares of NETAPP INC at a value of $24,710,816,000'},
 {'text': 'EDMP, INC. owns 4698 shares of NETAPP INC at a 

In [82]:
results = kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})
print(textwrap.fill(results[0]['text'], 60))

O'SHAUGHNESSY ASSET MANAGEMENT, LLC owns 5656 shares of
NETAPP INC at a value of $432,118,000


- Create a plain Question Answer chain
- Similarity search only, no augmentation by Cypher Query 

In [83]:
vector_store = Neo4jVector.from_existing_graph(
    embedding=VertexAIEmbeddings(model_name=EMBEDDING_MODEL),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)
# Create a retriever from the vector store
retriever = vector_store.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
plain_chain = RetrievalQAWithSourcesChain.from_chain_type(
    VertexAI(temperature=0), 
    chain_type="stuff", 
    retriever=retriever
)

- Create a second QA chain
- Augment similarity search using sentences found by the investment query above

In [84]:
investment_retrieval_query = """
MATCH (node)-[:PART_OF]->(f:Form),
    (f)<-[:FILED]-(com:Company),
    (com)<-[owns:OWNS_STOCK_IN]-(mgr:Manager)
WITH node, score, mgr, owns, com 
    ORDER BY owns.shares DESC LIMIT 10
WITH collect (
    mgr.managerName + 
    " owns " + owns.shares + 
    " shares in " + com.companyName + 
    " at a value of $" + 
    apoc.number.format(toInteger(owns.value)) + "." 
) AS investment_statements, node, score
RETURN apoc.text.join(investment_statements, "\n") + 
    "\n" + node.text AS text,
    score,
    { 
      source: node.source
    } as metadata
"""

In [85]:
vector_store_with_investment = Neo4jVector.from_existing_index(
    embedding=VertexAIEmbeddings(model_name=EMBEDDING_MODEL),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database="neo4j",
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY,
    retrieval_query=investment_retrieval_query,
)

# Create a retriever from the vector store
retriever_with_investments = vector_store_with_investment.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
investment_chain = RetrievalQAWithSourcesChain.from_chain_type(
    VertexAI(temperature=0), 
    chain_type="stuff", 
    retriever=retriever_with_investments
)

- Compare the outputs!

In [86]:
question = "In a single sentence, tell me about Netapp."

In [87]:
plain_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': ' NetApp is a global cloud-led, data-centric software company headquartered in San Jose, California.\n',
 'sources': ''}

In [88]:
investment_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': ' NetApp is a global cloud-led, data-centric software company headquartered in San Jose, California.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

- The LLM didn't make use of the investor information since the question didn't ask about investors
- Change the question and ask again

In [89]:
question = "In a single sentence, tell me about Netapp investors."

In [90]:
plain_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': ' NetApp investors are diversified across industry segments and vertical markets such as energy, financial services, government, technology, internet, life sciences, healthcare services, manufacturing, media, entertainment, animation, video postproduction, and telecommunications.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [91]:
investment_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': ' Vanguard Group Inc., BlackRock Inc., and PRIMECAP Management Co./CA/ are among the largest investors in NetApp Inc.\n',
 'sources': ''}

### Try for yourself

- Try changing the query above to retrieve other information
- Try asking different questions
- Note, if you change the Cypher query, you'll need to reset the retriever and QA chain